In [5]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

wf_md5 = "md5"
wf_fastqc = "fastqc-0-11-4-1"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md_qc = raw_input("Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)")

# status for completion
# there are two flavors of complete signals, before it was output_file_transfer_finished, not it is complete.
# old completed wf runs have former one.

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

################
##ADD TO WORKFLOW
# wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
# run_hours = int((datetime.now()-wfr_time).total_seconds()/3600)
################

def get_files(exp_set_id):
    files = []
    exps = ff_utils.get_metadata(exp_set_id, connection=ff)['experiments_in_set']
    for an_exp in exps:
        exp_resp = ff_utils.get_metadata(an_exp, connection=ff)['files']
        files.extend(exp_resp)
    return files

def summarize_file(file_resp):
    md5 = False
    qc = False
    file_id = file_resp['accession']
    sequencer = file_resp.get('instrument')
    relations = file_resp.get('related_files')
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    first_alias = file_resp.get('aliases',[None])[0]
    pair_no = file_resp.get('paired_end')
    # is md5 fine
    if file_resp.get('content_md5sum'):
        md5 = True
    # is there md5sum for gzip
    if not file_resp.get('md5sum'):
        print file_id,"does not have the md5sum calculated during upload"
        
    # is there a qc?
    if file_resp.get('quality_metric'):
        qc = True
    # Check workflows for qc fastqc workflow partA
    md5_status = 'did_not_run'
    fastqc_status = 'did_not_run'
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for a_wfr in workflows:
            wfr_resp = ff_utils.get_metadata(a_wfr['uuid'], connection=ff)
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_fastqc):
                fastqc_status = wfr_resp.get('run_status') 
            
                
    # Check for md5 and fastqc, and if not complete, run or report it. 
    # if exclude miseq is on, do this only if sequencer is not miseq

    if not md5 or status in ["uploading", "upload failed"] or md5_status != 'complete':
        # if not, shall we run it?
        if run_md_qc in ['md5', 'all']:
            print 'md5 running for', file_resp['accession']
            code_md5= "invoke run_md5 " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(3)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_resp['accession']
    # check fastqc if md5 is fine
    else:
        if not qc or fastqc_status != 'complete':
            # if not, shall we run it?
            if run_md_qc in ['qc', 'all']:
                print 'fastqc running for', file_resp['accession']
                code_qc= "invoke run_fastqc " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                run(code_qc)
                print ''    
                time.sleep(3)
            # user does not want it to be run, so just report
            else:
                print 'fastqc run missing for', file_resp['accession'], fastqc_status
                print 
    return


file_url = '/search/?type=FileFastq&limit=all&q=date_created%3A%3E%3D2016-09-01'
all_files = ff_utils.get_metadata('files-fastq', connection=ff)['@graph']

print len(all_files)


printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            #print "Deleted File", a_file
            continue
    except:
        print a_file
        break
        
    if counter-printn > 100:
        print counter
        printn = counter

    file_resp = ff_utils.get_metadata(a_file['uuid'], connection=ff, frame='embedded')
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket)
    if not head_info:
        print file_resp['accession'], "does not have a file in S3"
        continue
    file_info = summarize_file(file_resp)
    
 


Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)all
2755
fastqc running for 4DNFIXKHKJ1E
about to start run fastqc-0-11-4-1_4DNFIXKHKJ1E.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 3, 25, 8, 10, 54, 490000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '914ee528-3025-11e8-bffa-2ff892029bb9', 'HTTPHeaders': {'x-amzn-requestid': '914ee528-3025-11e8-bffa-2ff892029bb9', 'content-length': '145', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIXKHKJ1E.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIXKHKJ1E.fastq.gz

fastqc running for 4DNFI1CIZT87
about to start run fastqc-0-11-4-1_4DNFI1CIZT87.fastq.gz
response from aws was: 
 {u'startDate': datetime.datet

An error occurred (404) when calling the HeadObject operation: Not Found
4DNFI88R8KZ7 does not have a file in S3
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFI4IKEMGX does not have a file in S3
606
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFI5LI1ZLM does not have a file in S3
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFINBXXW77 does not have a file in S3
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFIY13CD93 does not have a file in S3
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFI43RP96B does not have a file in S3
An error occurred (404) when calling the HeadObject operation: Not Found
4DNFI8KTY3QD does not have a file in S3
4DNFIPBQPYG5 does not have the md5sum calculated during upload
4DNFIZUZHKUP does not have the md5sum calculated during upload
707
808
909


KeyboardInterrupt: 